In [46]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import ( hamming_loss, accuracy_score,
    f1_score, classification_report, recall_score, precision_score
)
import joblib

warnings.filterwarnings("ignore")
plt.style.use('bmh')

In [ ]:
def read_data(file_path):
    """
    Lee un archivo parquet y lo carga en un DataFrame de pandas.

    Parámetros
    ----------
    file_path: str
        Ruta al archivo parquet que se desea leer.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los datos cargados desde el archivo parquet.
    """
    data = pd.read_parquet(file_path)
    return data

In [45]:
def col_vectorizer_tfidf(df, col):
    """
    Convierte la columna especificada a string, la vectoriza usando TF-IDF y retorna la matriz y el vectorizador.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame que contiene la columna a vectorizar.
    col: str
        Nombre de la columna de texto.

    Retorna
    -------
    X : scipy.sparse.csr.csr_matrix
        Matriz TF-IDF.
    vectorizer : TfidfVectorizer
        Vectorizador entrenado.
    """
    df[col] = df[col].astype(str)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df[col].str.lower())
    return X, vectorizer

In [ ]:
def avg_jacard(y_true,y_pred):
    """
    Calcula el porcentaje promedio de similitud de Jaccard entre matrices binarias de etiquetas verdaderas y predichas.

    Parámetros
    ----------
    y_true : array-like de forma (n_muestras, n_etiquetas)
        Matriz binaria con las etiquetas verdaderas para cada muestra.
    y_pred : array-like de forma (n_muestras, n_etiquetas)
        Matriz binaria con las etiquetas predichas para cada muestra.

    Retorna
    -------
    float
        Porcentaje promedio de Jaccard (0–100) calculado como:
        (|intersección| / |unión|) medio por muestra, multiplicado por 100.
    """
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    return jacard.mean()*100

In [ ]:
def print_score(y_pred, clf):
    """
    Imprime métricas de evaluación de clasificación multilabel para un clasificador dado.

    Parámetros
    ----------
    y_pred : array-like
        Predicciones generadas por el clasificador.
    clf : objeto clasificador
        Instancia del modelo utilizado para predecir, se usa solo para mostrar su nombre de clase.

    Comportamiento
    -------------
    - Muestra por consola:
      * Nombre de la clase del clasificador.
      * Exactitud (accuracy).
      * Recall ponderado.
      * Precisión ponderada.
      * Puntuación F1 ponderada.
      * Puntuación de Jaccard promedio (porcentaje).
      * Hamming loss en porcentaje.
    - Asume que la variable global `y_test` contiene las etiquetas verdaderas.

    Retorna
    -------
    None
    """
    print("Clf: ", clf.__class__.__name__)
    print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))
    print("Recall score: {}".format(recall_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("Precision score: {}".format(precision_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("F1 score: {}".format(f1_score(y_pred, y_test, average='weighted')))
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")

In [32]:
data = pd.read_parquet('../data/output/StackOverflow.parquet')

In [34]:
X1, vectorizer = col_vectorizer_tfidf(data, 'Title')

In [35]:
X2, vectorizer = col_vectorizer_tfidf(data, 'Body')

In [36]:
y = data['Tag']
multilabel_binarizer = MultiLabelBinarizer()
new_y = multilabel_binarizer.fit_transform(y)

In [37]:
new_y

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1075316, 120))

In [38]:
X=hstack([X1,X2])

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size = 0.3, random_state = 42)

In [40]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(752721, 120))

In [41]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(752721, 3065030) (322595, 3065030) (752721, 120) (322595, 120)


In [ ]:
data['text'] = data['Title'] + " " + data['Body']

In [ ]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['Tag'])

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(data['text'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
print("F1-score (micro):", f1_score(y_test, y_pred, average='micro'))
print("F1-score (macro):", f1_score(y_test, y_pred, average='macro'))
print("\nReporte por etiqueta:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

In [43]:
print_score(y_pred, clf)

F1-score (micro): 0.6326671420079271
F1-score (macro): 0.5129166983827547

Reporte por etiqueta:
                    precision    recall  f1-score   support

         .htaccess       0.88      0.77      0.82      1042
              .net       0.52      0.09      0.16      4836
    actionscript-3       0.89      0.53      0.66       824
              ajax       0.73      0.49      0.59      3151
         algorithm       0.76      0.35      0.48      1317
           android       0.96      0.86      0.90     18231
    android-layout       0.49      0.11      0.18       796
         angularjs       0.95      0.82      0.88      4080
            apache       0.68      0.41      0.51      1314
               api       0.43      0.13      0.20       936
            arrays       0.61      0.31      0.41      3852
           asp.net       0.80      0.47      0.59      6064
       asp.net-mvc       0.68      0.43      0.53      2866
     asp.net-mvc-3       0.60      0.21      0.31       714
  

In [44]:
new_title = "Que es python y como lo podria integrar con SQL"
new_body = "No entiendo como se hace en sql un update"
new_text = new_title.lower() + ' ' + new_body.lower()
X_new = vectorizer.transform([new_text])
y_new_pred = clf.predict(X_new)
predicted_tags = mlb.inverse_transform(y_new_pred)
print("Etiquetas predichas para el nuevo texto:", predicted_tags)

Etiquetas predichas para el nuevo texto: [('python', 'sql')]


In [48]:

joblib.dump(clf, '../models/modelo_logreg.joblib')
joblib.dump(vectorizer, '../models/vectorizer_tfidf.joblib')
joblib.dump(mlb, '../models/multilabel_binarizer.joblib')

['../models/multilabel_binarizer.joblib']